## 0. Dependencies

In [37]:
# 5. Import the required modules

import networkx as nx
import nx_arangodb as nxadb

from arango import ArangoClient

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
import re

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool

import os
import networkx as nx
import nx_arangodb as nxadb

os.environ["DATABASE_HOST"] = "https://b61c3b83bfe6.arangodb.cloud:8529"
os.environ["DATABASE_USERNAME"] = "root"
os.environ["DATABASE_PASSWORD"] = "RHr0KzkRUVlp61IisH8G"
os.environ["DATABASE_NAME"] = "DAC_devops_log"

db = ArangoClient(hosts="https://b61c3b83bfe6.arangodb.cloud:8529") \
    .db(username="root", 
        password="RHr0KzkRUVlp61IisH8G", 
        name="DAC_devops_log",
        verify=True)

arango_graph = ArangoGraph(db)

G = nxadb.Graph(name="sprint1")

[22:06:01 -0500] [INFO]: Graph 'sprint1' exists.
[22:06:01 -0500] [INFO]: Default node type set to 'employee'


In [38]:
print(arango_graph.schema)

{'Graph Schema': [{'graph_name': 'MyGraph', 'edge_definitions': [{'edge_collection': 'MyGraph_node_to_MyGraph_node', 'from_vertex_collections': ['MyGraph_node'], 'to_vertex_collections': ['MyGraph_node']}]}, {'graph_name': 'sprint1', 'edge_definitions': [{'edge_collection': 'emp_adv_task', 'from_vertex_collections': ['employee'], 'to_vertex_collections': ['task']}, {'edge_collection': 'emp_do_task', 'from_vertex_collections': ['employee'], 'to_vertex_collections': ['task']}, {'edge_collection': 'task_precede_task', 'from_vertex_collections': ['task'], 'to_vertex_collections': ['task']}]}], 'Collection Schema': [{'collection_name': 'task_precede_task', 'collection_type': 'edge', 'edge_properties': [{'name': '_key', 'type': 'str'}, {'name': '_id', 'type': 'str'}, {'name': '_from', 'type': 'str'}, {'name': '_to', 'type': 'str'}, {'name': '_rev', 'type': 'str'}], 'example_edge': {'_key': '10010471', '_id': 'task_precede_task/10010471', '_from': 'task/TSK-98C9448B', '_to': 'task/TSK-5C2187C

In [ ]:
# from langchain_core.prompts import PromptTemplate

# AQL_GENERATION_TEMPLATE = """Task: Generate an ArangoDB Query Language (AQL) query from a User Input.

# You are an ArangoDB Query Language (AQL) expert responsible for translating a `User Input` into an ArangoDB Query Language (AQL) query.

# You are given an `ArangoDB Schema`. It is a JSON Object containing:
# 1. `Graph Schema`: Lists all Graphs within the ArangoDB Database Instance, along with their Edge Relationships.
# 2. `Collection Schema`: Lists all Collections within the ArangoDB Database Instance, along with their document/edge properties and a document/edge example.

# You may also be given a set of `AQL Query Examples` to help you create the `AQL Query`. If provided, the `AQL Query Examples` should be used as a reference, similar to how `ArangoDB Schema` should be used.

# Things you should do:
# - Think step by step.
# - Rely on `ArangoDB Schema` and `AQL Query Examples` (if provided) to generate the query.
# - Begin the `AQL Query` by the `WITH` AQL keyword to specify all of the ArangoDB Collections required.
# - Return the `AQL Query` wrapped in 3 backticks (```).
# - Use only the provided relationship types and properties in the `ArangoDB Schema` and any `AQL Query Examples` queries.
# - Only answer to requests related to generating an AQL Query.
# - If a request is unrelated to generating AQL Query, say that you cannot help the user.

# Things you should not do:
# - Do not use any properties/relationships that can't be inferred from the `ArangoDB Schema` or the `AQL Query Examples`. 
# - Do not include any text except the generated AQL Query.
# - Do not provide explanations or apologies in your responses.
# - Do not generate an AQL Query that removes or deletes any data.

# Under no circumstance should you generate an AQL Query that deletes any data whatsoever.

# ArangoDB Schema:
# {adb_schema}


# AQL Query: 
# """
# aql_prompt = PromptTemplate(
#     input_variables=["adb_schema"],
#     template=AQL_GENERATION_TEMPLATE,
# )

# aql_prompt.format(adb_schema=arango_graph.schema)


"Task: Generate an ArangoDB Query Language (AQL) query from a User Input.\n\nYou are an ArangoDB Query Language (AQL) expert responsible for translating a `User Input` into an ArangoDB Query Language (AQL) query.\n\nYou are given an `ArangoDB Schema`. It is a JSON Object containing:\n1. `Graph Schema`: Lists all Graphs within the ArangoDB Database Instance, along with their Edge Relationships.\n2. `Collection Schema`: Lists all Collections within the ArangoDB Database Instance, along with their document/edge properties and a document/edge example.\n\nYou may also be given a set of `AQL Query Examples` to help you create the `AQL Query`. If provided, the `AQL Query Examples` should be used as a reference, similar to how `ArangoDB Schema` should be used.\n\nThings you should do:\n- Think step by step.\n- Rely on `ArangoDB Schema` and `AQL Query Examples` (if provided) to generate the query.\n- Begin the `AQL Query` by the `WITH` AQL keyword to specify all of the ArangoDB Collections requ

## Example 1: basic query

In [39]:
@tool
def text_to_aql_to_text(query: str):
    """This tool is available to invoke the
    ArangoGraphQAChain object, which enables you to
    translate a Natural Language Query into AQL, execute
    the query, and translate the result back into Natural Language.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    chain = ArangoGraphQAChain.from_llm(
    	llm=llm,
    	graph=arango_graph,
    	verbose=True,
        allow_dangerous_requests=True,
        # aql_generation_prompt=aql_prompt

    )
    
    result = chain.invoke(query)

    return str(result["result"])

In [41]:
text_to_aql_to_text("what is the most important task in my graph?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH task
FOR t IN task
SORT t.StoryPoints DESC
LIMIT 1
RETURN t

AQL Result:
[{'_key': 'TSK-BBA87B09', '_id': 'task/TSK-BBA87B09', '_rev': '_jSntHIq--6', 'TaskID': 'TSK-BBA87B09', 'Description': 'Configuring server for scalability', 'AssignedEmployees': '33', 'Advisors': '27', 'PrecedingTasks': '', 'StoryPoints': '9', 'StartTime': '2025-03-18 00:00:00', 'EstimatedFinishTime': '2025-03-26 00:00:00', 'Status': 'Planned', 'ActualFinishTime': ''}]

> Finished chain.


'The most important task in your graph, based on the highest number of story points, is "Configuring server for scalability." This task is identified by the TaskID "TSK-BBA87B09" and is currently in the "Planned" status. It is scheduled to start on March 18, 2025, and is estimated to be completed by March 26, 2025. The task is assigned to employee 33 and has advisor 27.'

In [42]:
text_to_aql_to_text("what is the task that has the highest number of preceding tasks in my graph?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH task, task_precede_task
FOR t IN task
    LET precedingCount = LENGTH(
        FOR tp IN task_precede_task
            FILTER tp._to == t._id
            RETURN tp
    )
    SORT precedingCount DESC
    LIMIT 1
    RETURN { task: t, precedingCount: precedingCount }

AQL Result:
[{'task': {'_key': 'TSK-B03EF789', '_id': 'task/TSK-B03EF789', '_rev': '_jSntHIq-_F', 'TaskID': 'TSK-B03EF789', 'Description': 'Perform full data migration dry run', 'AssignedEmployees': '23', 'Advisors': '3,5', 'PrecedingTasks': 'TSK-42E44473,TSK-F567DBE1,TSK-A0500206,TSK-5A434477', 'StoryPoints': '13', 'StartTime': '2025-03-07 00:00:00', 'EstimatedFinishTime': '2025-03-14 00:00:00', 'Status': 'Planned', 'ActualFinishTime': ''}, 'precedingCount': 4}]

> Finished chain.


'The task with the highest number of preceding tasks in your graph is "Perform full data migration dry run" (Task ID: TSK-B03EF789). This task has four preceding tasks, indicating that it is dependent on the completion of these tasks before it can begin. The task is currently planned to start on March 7, 2025, and is estimated to finish by March 14, 2025.'

## 2: Example PageRank

In [43]:
@tool
def text_to_nx_algorithm_to_text(query):
    """This tool is available to invoke a NetworkX Algorithm on
    the ArangoDB Graph. You are responsible for accepting the
    Natural Language Query, establishing which algorithm needs to
    be executed, executing the algorithm, and translating the results back
    to Natural Language, with respect to the original query.

    If the query (e.g traversals, shortest path, etc.) can be solved using the Arango Query Language, then do not use
    this tool.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    ######################
    print("1) Generating NetworkX code")

    text_to_nx = llm.invoke(f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have the following graph analysis query: {query}.

    Generate the Python Code required to answer the query using the `G_adb` object.

    Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

    Only assume that networkx is installed, and other base python dependencies.

    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Make sure that `FINAL_RESULT` stores a short & consice answer. Avoid setting this variable to a long sequence.

    Your code:
    """).content

    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()
    
    print('-'*10)
    print(text_to_nx_cleaned)
    print('-'*10)

    ######################

    print("\n2) Executing NetworkX code")
    global_vars = {"G_adb": G, "nx": nx}
    local_vars = {}

    try:
        exec(text_to_nx_cleaned, global_vars, local_vars)
        text_to_nx_final = text_to_nx
    except Exception as e:
        print(f"EXEC ERROR: {e}")
        return f"EXEC ERROR: {e}"

        # TODO: Consider experimenting with a code corrector!
        attempt = 1
        MAX_ATTEMPTS = 3

        # while attempt <= MAX_ATTEMPTS
            # ...

    print('-'*10)
    FINAL_RESULT = local_vars["FINAL_RESULT"]
    print(f"FINAL_RESULT: {FINAL_RESULT}")
    print('-'*10)

    ######################

    print("3) Formulating final answer")

    nx_to_text = llm.invoke(f"""
        I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

        I have the following graph analysis query: {query}.

        I have executed the following python code to help me answer my query:

        ---
        {text_to_nx_final}
        ---

        The `FINAL_RESULT` variable is set to the following: {FINAL_RESULT}.

        Based on my original Query and FINAL_RESULT, generate a short and concise response to
        answer my query.
        
        Your response:
    """).content

    return nx_to_text


In [46]:
text_to_nx_algorithm_to_text("which task should be the most important and should be completed first if you are to use page rank?")

1) Generating NetworkX code
----------
import networkx as nx

# Assuming G_adb is already defined and is a NetworkX DiGraph

# Step 1: Calculate PageRank for the tasks
pagerank_scores = nx.pagerank(G_adb)

# Step 2: Filter out the tasks from the PageRank scores
task_scores = {node: score for node, score in pagerank_scores.items() if node.startswith('task/')}

# Step 3: Find the task with the highest PageRank score
most_important_task = max(task_scores, key=task_scores.get)

# Step 4: Extract the task ID from the node identifier
FINAL_RESULT = most_important_task.split('/')[1]
----------

2) Executing NetworkX code


[22:18:17 -0500] [INFO]: Graph 'sprint1' load took 2.423762083053589s


----------
FINAL_RESULT: TSK-AE8CFDEC
----------
3) Formulating final answer


'The task identified as the most important and should be completed first, based on PageRank analysis, is TSK-AE8CFDEC.'

## 3. Employee Importance

In [53]:
text_to_nx_algorithm_to_text("""
I need to understand the importance of my employees
                             
Help me create this output:
Employee | employee title | total pagerank of all the tasks this employee completed
""")

1) Generating NetworkX code
----------
import networkx as nx

# Assuming G_adb is already defined and is a NetworkX graph

# Step 1: Calculate PageRank for tasks
task_nodes = [n for n, d in G_adb.nodes(data=True) if d.get('collection') == 'task']
pagerank = nx.pagerank(G_adb, personalization=None, max_iter=100, tol=1e-06, nstart=None, weight='weight', dangling=None)

# Step 2: Filter PageRank values for task nodes
task_pagerank = {node: rank for node, rank in pagerank.items() if node in task_nodes}

# Step 3: Create a mapping of employees to tasks they completed
emp_do_task_edges = [(u, v) for u, v, d in G_adb.edges(data=True) if d.get('collection') == 'emp_do_task']
employee_task_map = {}
for emp, task in emp_do_task_edges:
    if emp not in employee_task_map:
        employee_task_map[emp] = []
    employee_task_map[emp].append(task)

# Step 4: Calculate total PageRank for tasks completed by each employee
employee_pagerank = {}
for emp, tasks in employee_task_map.items():
    total_p

"It seems that the `FINAL_RESULT` is empty, which indicates that no employees have completed any tasks, or there might be an issue with the data or the code logic. Here are a few things to check:\n\n1. **Data Verification**: Ensure that the `G_adb` graph is correctly populated with nodes and edges, especially the `emp_do_task` edges that link employees to tasks.\n\n2. **Collection Attribute**: Verify that the nodes and edges have the correct 'collection' attribute set, as the code relies on this to filter tasks and employee-task relationships.\n\n3. **Task Completion**: Confirm that there are tasks marked as completed by employees in the `emp_do_task` collection.\n\n4. **Code Logic**: Double-check the code logic to ensure that it correctly maps employees to tasks and calculates the PageRank for tasks.\n\nIf everything is set up correctly, the `FINAL_RESULT` should list employees, their titles, and the total PageRank of tasks they have completed. If issues persist, consider debugging th

## 4. Create schedule

In [ ]:
text_to_nx_algorithm_to_text("""
    Ignore start date and end date of any task
                             
    I will now need to set up a timeline to do all these tasks. I believe it can be done with respect to these conditions:
    - Using page rank, more important tasks should be done first
    - A task can only be done if all of its preceding tasks are done
                             
    Give me the order of all the tasks I need to do. Here is how I imagine you should do it:
    - For each task, calculate its pagerank
    - Calculate the order of tasks based on
    
""")

1) Generating NetworkX code
----------
import networkx as nx

# Assuming G_adb is already defined and is a NetworkX DiGraph

# Step 1: Calculate PageRank for all tasks
pagerank = nx.pagerank(G_adb)

# Step 2: Extract tasks and their PageRank scores
tasks_pagerank = {node: score for node, score in pagerank.items() if node.startswith('task/')}

# Step 3: Sort tasks by PageRank score in descending order
sorted_tasks = sorted(tasks_pagerank, key=tasks_pagerank.get, reverse=True)

# Step 4: Create a subgraph with only task nodes and task_precede_task edges
task_nodes = [node for node in G_adb.nodes if node.startswith('task/')]
task_edges = [(u, v) for u, v in G_adb.edges if u.startswith('task/') and v.startswith('task/')]
task_graph = nx.DiGraph()
task_graph.add_nodes_from(task_nodes)
task_graph.add_edges_from(task_edges)

# Step 5: Perform a topological sort on the task graph to respect task dependencies
topological_order = list(nx.topological_sort(task_graph))

# Step 6: Combine PageRank 

[22:28:30 -0500] [INFO]: Graph 'sprint1' load took 2.4557292461395264s


----------
FINAL_RESULT: ['task/TSK-AE8CFDEC', 'task/TSK-7E9F09A9', 'task/TSK-F7E19B69', 'task/TSK-B03EF789', 'task/TSK-5C2187CF', 'task/TSK-29CE08AD', 'task/TSK-98C9448B', 'task/TSK-F51C17B5', 'task/TSK-1FE396B7', 'task/TSK-FBD87AD9', 'task/TSK-763CE1C3', 'task/TSK-59CE546C', 'task/TSK-A0500206', 'task/TSK-8CD3CA3E', 'task/TSK-E73A7D25', 'task/TSK-01D25D85', 'task/TSK-F567DBE1', 'task/TSK-C2D38D23', 'task/TSK-DFB54467', 'task/TSK-5D974201', 'task/TSK-AAF06E6A', 'task/TSK-2B1FAB57', 'task/TSK-1B43CB90', 'task/TSK-FD3B1EFF', 'task/TSK-D5031D4A', 'task/TSK-89BBB564', 'task/TSK-983C30DC', 'task/TSK-4A3587C9', 'task/TSK-B600DD62', 'task/TSK-3D21853C', 'task/TSK-89806FDF', 'task/TSK-E6CFDD02', 'task/TSK-427652F3', 'task/TSK-5A434477', 'task/TSK-3BA37747', 'task/TSK-42E44473', 'task/TSK-0E209EFC', 'task/TSK-CB0C0404', 'task/TSK-FEA63216', 'task/TSK-85D5D00C', 'task/TSK-89754CBF', 'task/TSK-BE9555AC', 'task/TSK-FAAA4D51', 'task/TSK-84305F2F', 'task/TSK-BE4188D4', 'task/TSK-A6994DD6', 'task/TS

'Based on the analysis, the tasks should be completed in the following order, respecting both their importance (as determined by PageRank) and their dependencies (ensuring all preceding tasks are completed first):\n\n1. TSK-AE8CFDEC\n2. TSK-7E9F09A9\n3. TSK-F7E19B69\n4. TSK-B03EF789\n5. TSK-5C2187CF\n6. TSK-29CE08AD\n7. TSK-98C9448B\n8. TSK-F51C17B5\n9. TSK-1FE396B7\n10. TSK-FBD87AD9\n11. TSK-763CE1C3\n12. TSK-59CE546C\n13. TSK-A0500206\n14. TSK-8CD3CA3E\n15. TSK-E73A7D25\n16. TSK-01D25D85\n17. TSK-F567DBE1\n18. TSK-C2D38D23\n19. TSK-DFB54467\n20. TSK-5D974201\n21. TSK-AAF06E6A\n22. TSK-2B1FAB57\n23. TSK-1B43CB90\n24. TSK-FD3B1EFF\n25. TSK-D5031D4A\n26. TSK-89BBB564\n27. TSK-983C30DC\n28. TSK-4A3587C9\n29. TSK-B600DD62\n30. TSK-3D21853C\n31. TSK-89806FDF\n32. TSK-E6CFDD02\n33. TSK-427652F3\n34. TSK-5A434477\n35. TSK-3BA37747\n36. TSK-42E44473\n37. TSK-0E209EFC\n38. TSK-CB0C0404\n39. TSK-FEA63216\n40. TSK-85D5D00C\n41. TSK-89754CBF\n42. TSK-BE9555AC\n43. TSK-FAAA4D51\n44. TSK-84305F2F\n